# Scaling up

Originally released as part of a homework in ADA2019.

## Description

[Reddit](https://www.reddit.com/) aka *'the front page of the internet'* is a network of over a million *communities* aka *'subreddits'*, each of which covers a different topic based on people's interests. In other words, it is a *massive* collection of forums (corresponding to the aforementioned communities), where people can share content specific to a given topic or comment on other people’s posts.   

You are reddit's community manager and want to *appoint new moderators*. Because moderating a specific subreddit isn't a full-time job, you want the chosen moderators to moderate multiple subreddits at the same time. To make this choice effective, the moderators shouldn't have to spend too much time getting to know the community and the prevalent communication style, so it makes sense to let moderators moderate subreddits that are similar in communication style and language. At the same time, it also makes sense to let them moderate subreddits that are similar with respect to the participating users, because this allows moderators to track the behavior of individual users over multiple subreddits. For example, some users might only post offensive content once a month on a given subreddit, and therefore fly under the radar with someone moderating only that subreddit. However, considering all the subreddits these users post to, they might post something offensive every day but on different subreddits. Thus, a moderator in charge of all these subreddits would be able to ban such users much more effectively. In the light of the above description, your task is to find out ways to choose moderators considering both the textual content and the users of a subreddit.

### Dataset:
The dataset provided to you includes all the posts of the 15 largest subreddits written as of May 2015.

Reddit posts (provided to you via a [google drive folder](https://drive.google.com/a/epfl.ch/file/d/19SVHKbUTUPtC9HMmADJcAAIY1Xjq6WFv/view?usp=sharing))
```
reddit_posts
 |-- id: id of the post 
 |-- author: user name of the author 
 |-- body: text of the message
 |-- subreddit: name of the subreddit
```

Reddit scores (provided to you via a [google drive folder](https://drive.google.com/a/epfl.ch/file/d/1vr4PolJzTXr6ODSe3ucib5EAyp3rjxec/view?usp=sharing))
```
reddit_scores
 |-- id: id of the post 
 |-- score: score computed as sum of UP/DOWN votes
```

*Note: Jaccard similarity between subreddits represented using either the set of top-1000 words or the set of users can be computed locally (on the driver), however, all the other tasks have to be implemented in Spark.*

## B1. Getting a sense of the data

Start a PySpark instance...

In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *
import numpy as np 
import pandas as pd

conf = pyspark.SparkConf().setMaster("local[*]").setAll([
                                   ('spark.executor.memory', '12g'),  # find
                                   ('spark.driver.memory','4g'), # your
                                   ('spark.driver.maxResultSize', '2G') # setup
                                  ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

# FIX for Spark 2.x
locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

spark

... and load the data in a Spark dataframe.

In [2]:
rd_messages = spark.read.json("messages.json.gz")
rd_score = spark.read.json("score.json.gz")

In [3]:
posts_merged = rd_messages.join(rd_score, on="id", how="inner") 

In [4]:
posts_merged.show()  

+-------+-------------------+--------------------+---------------+-----+
|     id|             author|                body|      subreddit|score|
+-------+-------------------+--------------------+---------------+-----+
|cqug955|       _TimDuncan21|What did the 5 fi...|            nba|    1|
|cqug9j7|           Blakwulf|Not what i meant,...|           pics|    4|
|cqug9t1|           GheeGhee|      Giggle fart!!!|            nba|    1|
|cqug9yz|            bwishey|More like that aw...|          funny|    2|
|cquga10|       thricethefun|Bulls in 6 over Cavs|            nba|   20|
|cquga1r|   JakeCameraAction|IT WAS MINOR AND ...|         hockey|    3|
|cquga3k|         PhoKingGr8|/r/birdswitharms ...|         videos|    8|
|cquga53|   neverhaveiever23|                 LOL|            nfl|    1|
|cquga9q|  godless_communism|That's a hell of ...|      worldnews|    1|
|cqugaaj|    Hugh_G_Wrection|    Pathetic whistle|            nba|    0|
|cqugaba|    LastNewtStandin|I was calling it ...| 

### B1.1. Identify the most active subreddit

Print the list of subreddits along with the following information:
1. The total number of posts
2. The number of users with at least 1 message
3. The mean message length

*Note: Keep everything in one single dataframe and print the list sorted by number of posts in descending order.*

In [5]:
total_counts = posts_merged.count()
print(f"In total there are {total_counts:_} posts")

In total there are 7_984_080 posts


In [6]:
posts_count = posts_merged.groupBy("author").agg(
    count("*").alias("post_count"),
    avg(length("body")).alias("post_avg")
    ).sort(desc("post_count"))
posts_count.show()

+-------------------+----------+------------------+
|             author|post_count|          post_avg|
+-------------------+----------+------------------+
|      AutoModerator|     31403| 696.4621214533644|
|   imgurtranscriber|      4775|450.87916230366494|
|          Dancatpro|      3499|  82.6881966276079|
|TweetsInCommentsBot|      3325| 529.2508270676692|
|    sufficiency_bot|      2379|1109.1803278688524|
|       idealreaddit|      2216| 50.29061371841155|
|     GhostifiedMark|      2141| 51.10929472209248|
|    MisterWoodhouse|      2109|203.47226173541964|
|     Tower_Guardian|      1973|113.36746071971616|
|           wwxxyyzz|      1967|  87.8403660396543|
|       heat_forever|      1936|118.97107438016529|
|   Classic_Griswald|      1907| 281.0283167278448|
|           GRiZZY19|      1903| 139.4561219127693|
|        TweetPoster|      1860|             809.8|
|            chosena|      1855|154.38059299191374|
|          TrollaBot|      1845| 990.6926829268293|
|  Aperture-

In [7]:
mean_post_avg = posts_count.select(mean("post_avg")).collect()[0][0]
mean_post_avg

128.52622051302217

### B1.2. Identify the largest subreddit

Print *two* different lists of subreddits: ordered by (1) the number of posts, and (2) the number of users. For each subreddit, print the name and the corresponding counts.

Additionally, (3) plot the mean of message length for each subreddit in descending order.

In [8]:
subredit_post_counts = posts_merged.groupBy("subreddit").agg(
    countDistinct("author").alias("author_count"),
    count("*").alias("post_count"),
    avg(length("body")).alias("post_avg"),
    ).sort(desc("author_count")).cache()
subredit_post_counts.show()

+---------------+------------+----------+------------------+
|      subreddit|author_count|post_count|          post_avg|
+---------------+------------+----------+------------------+
|          funny|      224077|    691139|106.82283882113438|
|           pics|      205305|    564502| 114.9710045314277|
|         videos|      157628|    511492|170.22702603364274|
|leagueoflegends|      119321|   1151287|152.72280760574904|
|  AdviceAnimals|      115815|    411902| 159.2513801826648|
|      worldnews|       99261|    439417|224.93754679495785|
|           news|       98736|    477658| 230.9491602778557|
|         movies|       92484|    354601|164.83209297210104|
|GlobalOffensive|       46686|    382017| 147.2883981602913|
|            nba|       45034|    704862|106.48656758344187|
|         soccer|       41648|    455215|134.42224663071295|
|            nfl|       41593|    534345|148.96989211090212|
|          DotA2|       41466|    445154|141.48906670500546|
| DestinyTheGame|       

In [9]:
subredit_post_counts = posts_merged.groupBy("subreddit").agg(
    countDistinct("author").alias("author_count"),
    count("*").alias("post_count"),
    avg(length("body")).alias("post_avg"),
    ).sort(desc("post_count"))
subredit_post_counts.show()

+---------------+------------+----------+------------------+
|      subreddit|author_count|post_count|          post_avg|
+---------------+------------+----------+------------------+
|leagueoflegends|      119321|   1151287|152.72280760574904|
|            nba|       45034|    704862|106.48656758344187|
|          funny|      224077|    691139|106.82283882113438|
|           pics|      205305|    564502| 114.9710045314277|
|            nfl|       41593|    534345|148.96989211090212|
|         videos|      157628|    511492|170.22702603364274|
|           news|       98736|    477658| 230.9491602778557|
| DestinyTheGame|       37008|    471160|165.41786866457255|
|         soccer|       41648|    455215|134.42224663071295|
|          DotA2|       41466|    445154|141.48906670500546|
|      worldnews|       99261|    439417|224.93754679495785|
|  AdviceAnimals|      115815|    411902| 159.2513801826648|
|         hockey|       25568|    389329| 95.37287230080472|
|GlobalOffensive|       

In [10]:
subredit_post_counts = posts_merged.groupBy("subreddit").agg(
    countDistinct("author").alias("author_count"),
    count("*").alias("post_count"),
    avg(length("body")).alias("post_avg"),
).sort(desc("post_avg"))
subredit_post_counts.show()

+---------------+------------+----------+------------------+
|      subreddit|author_count|post_count|          post_avg|
+---------------+------------+----------+------------------+
|           news|       98736|    477658| 230.9491602778557|
|      worldnews|       99261|    439417|224.93754679495785|
|         videos|      157628|    511492|170.22702603364274|
| DestinyTheGame|       37008|    471160|165.41786866457255|
|         movies|       92484|    354601|164.83209297210104|
|  AdviceAnimals|      115815|    411902| 159.2513801826648|
|leagueoflegends|      119321|   1151287|152.72280760574904|
|            nfl|       41593|    534345|148.96989211090212|
|GlobalOffensive|       46686|    382017| 147.2883981602913|
|          DotA2|       41466|    445154|141.48906670500546|
|         soccer|       41648|    455215|134.42224663071295|
|           pics|      205305|    564502| 114.9710045314277|
|          funny|      224077|    691139|106.82283882113438|
|            nba|       

### B1.3. Identify the subreddit with the highest average score

Print the list of subreddits sorted by their average content scores.

In [11]:
subredit_scores = posts_merged.groupby("subreddit").agg(
    avg("score").alias("avg_score")
).sort(desc("avg_score"))
subredit_scores.show()

+---------------+------------------+
|      subreddit|         avg_score|
+---------------+------------------+
|         videos|12.649445152612358|
|           pics|12.216559020162904|
|          funny|12.041505399058655|
|  AdviceAnimals|11.251695791717447|
|         soccer|10.634627593554693|
|         movies|  9.82014997137628|
|            nfl| 9.048348913155358|
|            nba| 9.032795071943161|
|           news| 8.673421150697779|
|      worldnews|  7.86683719564787|
|         hockey| 6.520120515039979|
|leagueoflegends| 5.983557531701479|
|          DotA2| 4.880537971129092|
|GlobalOffensive| 4.351442475073099|
| DestinyTheGame|3.0288819084811953|
+---------------+------------------+



## B2. Moderator assignment based on Subreddit Textual Content

Different subreddits follow different communication styles inherent in the topic and the community. Having said that, the goal is to discover similar subreddits by only looking at the *words* present in the posted messages. Once such a list of similar subreddits is identified, an appropriately chosen moderator can then be assigned to all these subreddits.

Specifically, the task boils down to computing a similarity score between two subreddits based on the *words* present in their textual content. Your first idea is to use the *Jaccard similarity*, which is defined as the size of the intersection of two sets divided by the size of their union.

$Jaccard(A,B) = \frac{|A \cap B|}{|A \cup B|}$

In [12]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    # return len(s1.intersection(s2)) / len(s1.union(s2))
    return len(s1 & s2) / len(s1 | s2)

### B2.1.
The first step requires constructing a set representation of each subreddit. The goal is to represent each subreddit as a *set of words* existing in the messages posted on that subreddit. Compute the 50,000 most frequent words across all the provided subreddits. Construct a representation for each subreddit by retaining only the words found in the previously identified set of 50,000 frequent words.

Some rules:
 * Words are defined as tokens matching the regular expression `\W`
 * Remove all the stop-words (English language)

*Note: You might find the [RegexTokenizer](https://spark.apache.org/docs/2.2.0/ml-features.html#tokenizer) and the [StopWordsRemover](https://spark.apache.org/docs/2.2.0/ml-features.html#stopwordsremover) utilities available in the package pyspark.ml useful for this task as they help you in transforming the features and removing stopwords.*

In [13]:
from pyspark.ml.feature import StopWordsRemover, Tokenizer

In [14]:
tokenizer = Tokenizer(inputCol="body", outputCol="body_tokens")
remover = StopWordsRemover(inputCol="body_tokens", outputCol="body_filtered")

In [15]:
posts_merged = tokenizer.transform(posts_merged)
posts_merged = remover.transform(posts_merged)
posts_merged.show()

+-------+-------------------+--------------------+---------------+-----+--------------------+--------------------+
|     id|             author|                body|      subreddit|score|         body_tokens|       body_filtered|
+-------+-------------------+--------------------+---------------+-----+--------------------+--------------------+
|cqug955|       _TimDuncan21|What did the 5 fi...|            nba|    1|[what, did, the, ...|[5, fingers, say,...|
|cqug9j7|           Blakwulf|Not what i meant,...|           pics|    4|[not, what, i, me...|[meant,, pretty, ...|
|cqug9t1|           GheeGhee|      Giggle fart!!!|            nba|    1|   [giggle, fart!!!]|   [giggle, fart!!!]|
|cqug9yz|            bwishey|More like that aw...|          funny|    2|[more, like, that...|[like, awkward, m...|
|cquga10|       thricethefun|Bulls in 6 over Cavs|            nba|   20|[bulls, in, 6, ov...|    [bulls, 6, cavs]|
|cquga1r|   JakeCameraAction|IT WAS MINOR AND ...|         hockey|    3|[it, was

### B2.2.
* Compute the Jaccard similarity between all the subreddits using the set representation obtained in step **B2.1.**, and plot in a heatmap the similarity values of all the pairs of subreddits.
* Analyze this plot and discuss your observations. Do you observe that subreddits corresponding to similar topics possess higher Jaccard similarity?
* Provide detailed interpretations of the obtained results. Specifically,
    - Explain the limitations of your conclusions, and discuss the potential reasons.
    - Explain the potential problems with the Jaccard similarity function.

In [16]:
posts_merged.select("subreddit").distinct().count()

15

In [17]:
all_words = posts_merged.select(explode("body_filtered").alias("word"))

top50k = all_words.groupby("word").agg(count("*").alias("total")).sort(col("total").desc()).limit(50_000)

top50k_set = set(row.word for row in top50k.collect())

In [18]:
top50k_broadcast = spark.sparkContext.broadcast(top50k_set)

# Create a UDF that uses the broadcast variable
@udf(returnType=ArrayType(StringType()))
def remove_words_filter_udf(text):
    if text is None:
        return []
    return list(set(text) & top50k_broadcast.value)

# Apply the function and keep other columns
posts_merged_50k = posts_merged.withColumn(
    "top_50k_words", 
    remove_words_filter_udf("body_filtered")
)

In [19]:
similarity = []
for sr1 in posts_merged_50k:
    for sr2 in posts_merged_50k:
        similarity.append((sr1[0], sr2[0], jaccard_similarity(sr1[1], sr2[1])))


similarity_matrix_50k_words = pd.DataFrame(similarity).pivot(index=0, columns=1, values=2)

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

### B2.3.

* Alternatively, compute the 1000 most frequent words for each subreddit, construct its representation as the set of top-1000 words, and print a heatmap with the Jaccard similarity like in step **B2.2.**.
* Explain your observations in detail: how and why is this new result different from the one obtained in **B2.2.**?

*Note: Use the same rules specified in B2.1: words tokenized with the regex \W and stop-words removed*

## B3. Moderator assignment based on Subreddit Users

Subreddits can be seen as communities of people interacting about a common topic. As an alternative to the *textual content* based similarity in **B2**, your task here is to validate if similarity between two subreddits can be measured based on their participating users.

Of course users are not monothematic, and they interact with multiple subreddits. In this task, we are specifically interested in observing the amount of overlap across different subreddits based on their participating users. Similar to **B2**, the overlap is measured using the *Jaccard similarity*.


### B3.1.
Construct a set representation of each subreddit as the users that posted at least one time in that subreddit.

Some users are very talkative and active across different topics. Print the username of the person that posted in the maximum number of subreddits. *Note that users who posted at least once in a subreddit are considered as participant of that subreddit.*

### B3.2.

* Compute the Jaccard similarity between all the subreddits using the set representation obtained in step **B3.1.**, and visualise it similar to **B2**.
* Analyze this plot, identify highly similar pairs of subreddits, and clearly describe your observations.

## B4. Language vs. Users similarity
    
* Visualize the similarity scores based on word (**B2.3.**) and user (**B3**) similarity on the x and y axes respectively for the subreddit `NBA` compared to all the other subreddits. Do some semantically meaningful groups emerge? Provide clear explanataions of your observations.
* Furthermore, do you observe differences in similarities between various sports-related subreddits in the dataset? Please provide explanations of the reasons behind these differences, if any!